# Bayesian Neural Network with Variational Inference
### Example: Boston Housing Dataset (Regression)

In [ ]:
import torch
import torch.optim as optim
from datasets.data_loader import load_dataset
from models.variational_inference import BayesianNN, elbo_loss
import matplotlib.pyplot as plt

In [ ]:
# Load dataset
train_loader, test_loader = load_dataset(
    csv_path="datasets/sample_data.csv",
    target_column="target",
    test_size=0.2,
    batch_size=32
)

In [ ]:
# Define model and optimizer
input_dim = next(iter(train_loader))[0].shape[1]
model = BayesianNN(input_dim=input_dim, hidden_dim=64, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Training loop
epochs = 100
train_losses = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        preds = model(x_batch)
        kl = model.kl_divergence()
        loss = elbo_loss(preds, y_batch, kl, beta=0.1)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

In [ ]:
# Plot training loss
plt.plot(train_losses)
plt.xlabel("Epoch")
plt.ylabel("ELBO Loss")
plt.title("Training Loss (Bayesian NN)")
plt.show()